In [31]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

# Scrape Billboard Top 100

In [74]:
base_url = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs/'
years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [144]:
### Scrape topp 100 songs by year
def get_songs(year):
    base_url = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs/'
    
    song_list = []
    page = requests.get(base_url.format(year))
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all(id='title-of-a-story')
    
    if year in (2011, 2016):
        for i in results[:99]:
            song_name = i.text.replace('\n', '').replace('\t','')
            song_list.append(song_name)
    else:
        for i in results[:100]:
            song_name = i.text.replace('\n', '').replace('\t','')
            song_list.append(song_name)
        
    return song_list

### Scrape artist names for songs
def get_artists(year):
    base_url = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs/'
    
    page = requests.get(base_url.format(year))
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all('span', class_='c-label')
    
    artist_list = [results[i].text.replace('\n', '').replace('\t','') for i in range(1, len(results)+1, 2)]
    
    return artist_list

In [145]:
### Aggregating data across all years
year_col= []
song_col = []
artist_col = []# 2011, 2016 missing 1 value each

for year in years: 
    if year in (2011, 2016):
        year_100 = np.zeros(99)+year
    else:
        year_100 = np.zeros(100)+year
    year_col = np.concatenate((year_col, year_100), axis=0) 
    
    songs = get_songs(year)
    song_col = np.concatenate((song_col, songs), axis=0)
    
    artists = get_artists(year)
    artist_col = np.concatenate((artist_col, artists), axis=0)

In [147]:
# Creating df
charts = dict()
charts['year'] = year_col
charts['song'] = song_col
charts['artist'] = artist_col 

charts_df = pd.DataFrame(charts)

In [148]:
charts_df.head()

,year,song,artist
0,2011.0,Rolling In The Deep,Adele
1,2011.0,Party Rock Anthem,LMFAO Featuring Lauren Bennett & GoonRock
2,2011.0,Firework,Katy Perry
3,2011.0,E.T.,Katy Perry Featuring Kanye West
4,2011.0,Give Me Everything,"Pitbull Featuring Ne-Yo, Afrojack & Nayer"


# Collect lyrics from Genius API